확률을 구할 때 ★ <br>
```python
rf_pred_tr = rf.predict_proba(X_tr)
roc_auc_score(y_tr, rf_pred_tr[:,1])
```
**roc_auc_score는 무조건 확률!!**

## 전자상거래 배송 데이터
### 제품 배송 시간에 맞춰 배송되었는지 예측모델 만들기
학습용 데이터 (X_train, y_train)을 이용하여 배송 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측 확률값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)
```
ID, Reached.on.Time_Y.N
4733,0.6
2040,0.8
5114,0.45
2361,0.23
5996,0.43
```

[시험용 데이터셋 만들기] 코드는 예시문제와 동일한 형태의 X_train, y_train, X_test 데이터를 만들기 위함임

(유의사항)
- 성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, 피처엔지니어링, 분류알고리즘, 하이퍼파라미터 튜닝, 모형 앙상블 등이 수반되어야 한다.
- 수험번호.csv파일이 만들어지도록 코드를 제출한다.
- 제출한 모델의 성능은 ROC-AUC형태로 읽어드린다.

## [참고]작업형2 문구
- 출력을 원하실 경우 print() 함수 활용
- 예시) print(df.head())
- getcwd(), chdir() 등 작업 폴더 설정 불필요
- 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

### 데이터 파일 읽기 예제
- import pandas as pd
- X_test = pd.read_csv("data/X_test.csv")
- X_train = pd.read_csv("data/X_train.csv")
- y_train = pd.read_csv("data/y_train.csv")

### 사용자 코딩

### 답안 제출 참고
- 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
- pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)

In [31]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("../input/customer-analytics/Train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((8799, 11), (2200, 11), (8799, 2), (2200, 2))

## 사용자 코딩

In [32]:
# 제공된 3개 데이터 (y_test데이터 활용X)
X_train.shape, X_test.shape, y_train.shape

((8799, 11), (2200, 11), (8799, 2))

In [33]:
display(X_train.head())
display(y_train.head())

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
3999,4000,B,Ship,3,4,134,3,high,F,6,5680
9559,9560,F,Ship,4,3,173,3,medium,M,5,5331
2649,2650,B,Ship,2,1,192,3,high,M,46,3206
4843,4844,F,Ship,6,5,284,4,medium,M,8,5346
9601,9602,F,Flight,3,1,246,3,low,F,10,4707


,ID,Reached.on.Time_Y.N
3999,4000,0
9559,9560,1
2649,2650,1
4843,4844,1
9601,9602,1


In [34]:
X_train.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [35]:
X_test.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [36]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# ID 컬럼 삭제
X_train_drop = X_train.drop(columns=['ID'])
X_test_drop = X_test.drop(columns=['ID'])

# 수치형/범주형 변수 전처리
v_col = ['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']
n_col = ['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms']

ss = StandardScaler()
le = LabelEncoder()

y = y_train['Reached.on.Time_Y.N']

ss.fit_transform(X_train_drop[n_col])
ss.transform(X_test_drop[n_col])

for i in v_col:
    X_train_drop[i] = le.fit_transform(X_train_drop[i])
    X_test_drop[i] = le.transform(X_test_drop[i])

# Train 데이터 분리
X_tr, X_val, y_tr, y_val = train_test_split(X_train_drop, y, test_size=0.25, random_state=42)
    
# 모델 학습 (랜덤 포레스트)
rf = RandomForestClassifier()
rf.fit(X_tr, y_tr)
rf_pred_tr = rf.predict_proba(X_tr)
print(roc_auc_score(y_tr, rf_pred_tr[:,1]))

rf_pred_val = rf.predict_proba(X_val)
print(roc_auc_score(y_val, rf_pred_val[:,1]))

# 모델 학습 (XGBoost)
xgb = XGBClassifier()
xgb.fit(X_tr, y_tr)
xgb_pred_tr = xgb.predict_proba(X_tr)
print(roc_auc_score(y_tr, xgb_pred_tr[:,1]))

xgb_pred_val = xgb.predict_proba(X_val)
print(roc_auc_score(y_val, xgb_pred_val[:,1]))

# 결과 export
pd.DataFrame({'ID':X_test.ID, 'Reached.on.Time_Y.N':rf.predict_proba(X_test_drop)[:,1]}).to_csv('000000000.csv', index=False)

1.0
0.7620612598728569
[10:28:21] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.9848828005820758
0.7488045548920139


In [37]:
result = pd.read_csv('./000000000.csv')
result.head()

,ID,Reached.on.Time_Y.N
0,4733,0.46
1,2040,1.00
2,5114,0.50
3,2361,0.97
4,5996,0.39


In [38]:
roc_auc_score(y_test['Reached.on.Time_Y.N'], rf.predict_proba(X_test_drop)[:,1])

0.7192237969676993

## 풀이 (Baseline)
- 아래코드는 정답이 아닌 풀이 예시입니다.

## EDA

In [39]:
# 데이터 확인
print(X_train.shape)
X_train.head()

(8799, 11)


,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
3999,4000,B,Ship,3,4,134,3,high,F,6,5680
9559,9560,F,Ship,4,3,173,3,medium,M,5,5331
2649,2650,B,Ship,2,1,192,3,high,M,46,3206
4843,4844,F,Ship,6,5,284,4,medium,M,8,5346
9601,9602,F,Flight,3,1,246,3,low,F,10,4707


In [40]:
y_train

,ID,Reached.on.Time_Y.N
3999,4000,0
9559,9560,1
2649,2650,1
4843,4844,1
9601,9602,1
...,...,...
3934,3935,0
2669,2670,1
1152,1153,1
6201,6202,0


In [41]:
# 레이블(타겟) 확인
y_train['Reached.on.Time_Y.N'].value_counts()

1    5251
0    3548
Name: Reached.on.Time_Y.N, dtype: int64

In [42]:
# X_train 결측치 확인
X_train.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [43]:
# X_test 결측치 확인
X_test.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [44]:
# 데이터 타입 확인
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8799 entries, 3999 to 9332
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   8799 non-null   int64 
 1   Warehouse_block      8799 non-null   object
 2   Mode_of_Shipment     8799 non-null   object
 3   Customer_care_calls  8799 non-null   int64 
 4   Customer_rating      8799 non-null   int64 
 5   Cost_of_the_Product  8799 non-null   int64 
 6   Prior_purchases      8799 non-null   int64 
 7   Product_importance   8799 non-null   object
 8   Gender               8799 non-null   object
 9   Discount_offered     8799 non-null   int64 
 10  Weight_in_gms        8799 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 824.9+ KB


In [45]:
# object 타입 컬럼, 고유값 개수 확인
X_train[['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']].nunique()

Warehouse_block       5
Mode_of_Shipment      3
Product_importance    3
Gender                2
dtype: int64

## 데이터 전처리

In [46]:
# object 컬럼 삭제 (또는 라벨인코딩, 원핫인코딩)
X_train = X_train.drop(['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender'], axis=1)
X_test = X_test.drop(['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender'], axis=1)
X_train

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
3999,4000,3,4,134,3,6,5680
9559,9560,4,3,173,3,5,5331
2649,2650,2,1,192,3,46,3206
4843,4844,6,5,284,4,8,5346
9601,9602,3,1,246,3,10,4707
...,...,...,...,...,...,...,...
3934,3935,6,5,178,3,5,4096
2669,2670,3,1,206,2,56,2069
1152,1153,4,1,135,10,35,1989
6201,6202,4,4,251,2,6,4376


## 모델 및 평가

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [48]:
X_train_id = X_train.pop('ID')
X_test_id = X_test.pop('ID')

In [49]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(X_train,y_train['Reached.on.Time_Y.N'],test_size=0.2,random_state=2021)

In [50]:
from sklearn.metrics import roc_auc_score

In [51]:
model = LogisticRegression()
model.fit(X_tr, y_tr)
pred = model.predict_proba(X_val)
roc_auc_score(y_val, pred[:,1])

0.7248437200995088

In [52]:
model = KNeighborsClassifier()
model.fit(X_tr, y_tr)
pred = model.predict_proba(X_val)
roc_auc_score(y_val, pred[:,1])

0.7321048882651868

In [53]:
model = DecisionTreeClassifier()
model.fit(X_tr, y_tr)
pred = model.predict_proba(X_val)
roc_auc_score(y_val, pred[:,1])

0.6114132168144415

In [54]:
model = RandomForestClassifier()
model.fit(X_tr, y_tr)
pred = model.predict_proba(X_val)
roc_auc_score(y_val, pred[:,1])

0.7351607184622908

In [55]:
model = XGBClassifier(eval_metric='mlogloss', use_label_encoder=False)
model.fit(X_tr, y_tr)
pred = model.predict_proba(X_val)
roc_auc_score(y_val, pred[:,1])

0.7335108758053199

## 모델 선택 및 결과 출력

In [56]:
model = KNeighborsClassifier()
model.fit(X_train, y_train['Reached.on.Time_Y.N'])
pred = model.predict_proba(X_test)
pred

array([[0.4, 0.6],
       [0. , 1. ],
       [0.8, 0.2],
       ...,
       [0.4, 0.6],
       [0.2, 0.8],
       [0.6, 0.4]])

In [57]:
submission = pd.DataFrame({
        "ID": X_test_id,
        "Reached.on.Time_Y.N": pred[:,1]
    })

In [58]:
submission.head()

,ID,Reached.on.Time_Y.N
4732,4733,0.6
2039,2040,1.0
5113,5114,0.2
2360,2361,1.0
5995,5996,0.6


In [59]:
submission.to_csv('submission.csv', index=False)  ## 수험번호.csv

## 결과 체점 (수험자는 알 수 없는 부분임)

In [60]:
# 검증데이터(val)보다 조금 떨어진 성능을 보여주고 있음
pred = model.predict_proba(X_test)
roc_auc_score(y_test['Reached.on.Time_Y.N'], pred[:,1])

0.7143635155735003